<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

In [3]:
import os
os.environ["REPROWORKDIR"] = "reproduce"

In [1]:
#| echo: false
#| output: asis
show_doc(requires_config)

---

### requires_config

>      requires_config (func)

In [2]:
#| echo: false
#| output: asis
show_doc(validate_base_config)

---

### validate_base_config

>      validate_base_config (base_config, quiet=False)

In [3]:
#| echo: false
#| output: asis
show_doc(ReproduceWorkEncoder)

---

### ReproduceWorkEncoder

>      ReproduceWorkEncoder (_dict=<class 'dict'>, preserve=False)

Initialize self.  See help(type(self)) for accurate signature.

In [4]:
#| echo: false
#| output: asis
show_doc(read_base_config)

---

### read_base_config

>      read_base_config ()

In [5]:
#| echo: false
#| output: asis
show_doc(set_default_dir)

---

### set_default_dir

>      set_default_dir (quiet=True)

In [6]:
#| echo: false
#| output: asis
show_doc(printrw)

---

### printrw

>      printrw (*args, **kwargs)

fancy reproduce.work print function

In [7]:
#| echo: false
#| output: asis
show_doc(test_validate_base_config)

---

### test_validate_base_config

>      test_validate_base_config ()

In [8]:
#| echo: false
#| output: asis
show_doc(generate_config)

---

### generate_config

>      generate_config (options={}, version='reproduce.work/v1/default')

In [9]:
#| echo: false
#| output: asis
show_doc(construct_config)

---

### construct_config

>      construct_config (options)

In [10]:
#| echo: false
#| output: asis
show_doc(ReproduceWorkEncoder)

---

### ReproduceWorkEncoder

>      ReproduceWorkEncoder (_dict=<class 'dict'>, preserve=False)

Initialize self.  See help(type(self)) for accurate signature.

In [6]:
base_config = {
        'authors': {
            'author1.name': 'Alex P. Miller',
            'author1.affiliation': 'USC Marshall School of Business',
            'author1.email': 'alex.miller@marshall.usc.edu'
        },
        'repro': {
            'version': 'reproduce.work/v1/default',
            'stages': ['init', 'develop', 'build']
        },
        'repro.files': {
            'input': 'document/main.md',
            'dynamic': 'document/pubdata.toml',
            'latex_template': 'document/latex/template.tex',
            'output': 'document/latex/compiled.tex'
        },
        'repro.stage.init': {
            'script': 'docker build -t {dev_image_tag} .\ndocker build -t tinytex {reproduce_dir}/Dockerfile.tinytex\ndocker build -t watcher {reproduce_dir}/Dockerfile.watch\n'
        },
        'repro.stage.develop': {
            'script': 'docker run -v $(pwd):/home/jovyan -p 8888:8888 {dev_image_tag} start.sh jupyter lab --LabApp.token=\'\'\n\\INSERT{watch_cmd_here}\n'
        },
        'repro.stage.build': {
            'script': 'docker run --rm -i -v $(pwd):/home/jovyan -p 8888:8888 {dev_image_tag} python reproduce_work.build() # this replaces instances of \\INSERT{var} in `input` file\ndocker run --rm -i -v $(pwd):/home lowdown # this converts the markdown to latex\ndocker run --rm -i --net=none -v $(pwd):/home tinytex sh -c "cd /home/document/latex && xelatex compiled.tex" # this compiles the latex\n'
        }
    }
validate_base_config(base_config)

True

In [11]:
#| echo: false
#| output: asis
show_doc(update_watched_files)

---

### update_watched_files

>      update_watched_files (add=[], remove=[], quiet=False)

In [12]:
#| echo: false
#| output: asis
show_doc(register_notebook)

---

### requires_config.<locals>.wrapper

>      requires_config.<locals>.wrapper (*args, **kwargs)

In [13]:
#| echo: false
#| output: asis
show_doc(publish_variable)

---

### publish_variable

>      publish_variable (value, var_name, metadata={})

In [14]:
#| echo: false
#| output: asis
show_doc(reproducible)

---

### reproducible

>      reproducible (var_assignment_func)

A decorator to register the line number and timestamp when a variable is assigned.

In [15]:
#| echo: false
#| output: asis
show_doc(publish_file)

---

### requires_config.<locals>.wrapper

>      requires_config.<locals>.wrapper (*args, **kwargs)

In [16]:
#| echo: false
#| output: asis
show_doc(publish_data)

---

### requires_config.<locals>.wrapper

>      requires_config.<locals>.wrapper (*args, **kwargs)

In [17]:
#| echo: false
#| output: asis
show_doc(ReproduceWorkEncoder)

---

### ReproduceWorkEncoder

>      ReproduceWorkEncoder (_dict=<class 'dict'>, preserve=False)

Initialize self.  See help(type(self)) for accurate signature.

In [18]:
#| echo: false
#| output: asis
show_doc(serialize_to_toml)

---

### serialize_to_toml

>      serialize_to_toml (data, root=True)

Unified function to serialize various Python data types to TOML format.

In [19]:
#| echo: false
#| output: asis
show_doc(check_for_defintion_in_context)

---

### check_for_defintion_in_context

>      check_for_defintion_in_context (function_name='save')

In [20]:
#| echo: false
#| output: asis
show_doc(get_cell_index)

---

### get_cell_index

>      get_cell_index ()

Get the current cell index in a Jupyter notebook environment.
If not in Jupyter, return None.

In [14]:
publish_data(67890, "test_var_timestamp_1")  # This should capture this line number and timestamp
publish_data("Hello again!", "test_var_timestamp_2")  # And this line number and timestamp

VAR_REGISTRY

{'p_value_str': {'description': 'The p-value of the coefficient on the slope of the linear regression line.', 'published_url': 'https://github.com/reproduce-work/reproduce-work/reproduce/pubdata.toml', 'generating_script': 'https://github.com/reproduce-work/reproduce-work/blob/main/nbs/00_introduction.ipynb', 'type': 'data', 'content_hash': 'd68a5a88e3ca0ee94ee5951c43fbb552', 'python_version': '3.9.13 | packaged by conda-forge | (main, May 27 2022, 17:23:23)  [GCC 10.3.0]', 'platform_info': 'Linux-6.4.16-orbstack-00105-g14094bfeec09-aarch64-with-glibc2.35', 'value': '0.068'}, 'x': {'description': 'The simulated X data', 'units': 'kilograms', 'published_url': 'https://github.com/reproduce-work/reproduce-work/reproduce/pubdata.toml', 'generating_script': 'https://github.com/reproduce-work/reproduce-work/blob/main/nbs/00_introduction.ipynb', 'type': 'data', 'content_hash': '38f13b81a58a7d931600e917d77dfe8f', 'python_version': '3.9.13 | packaged by conda-forge | (main, May 27 2022, 17:23:2

{'REPROWORK_REMOTE_URL': None, 'REPROWORK_ACTIVE_NOTEBOOK': None}

In [11]:
publish_data(np.array([1,2,3]), 'array')

{'p_value_str': {'description': 'The p-value of the coefficient on the slope of the linear regression line.', 'published_url': 'https://github.com/reproduce-work/reproduce-work/reproduce/pubdata.toml', 'generating_script': 'https://github.com/reproduce-work/reproduce-work/blob/main/nbs/00_introduction.ipynb', 'type': 'data', 'content_hash': 'd68a5a88e3ca0ee94ee5951c43fbb552', 'python_version': '3.9.13 | packaged by conda-forge | (main, May 27 2022, 17:23:23)  [GCC 10.3.0]', 'platform_info': 'Linux-6.4.16-orbstack-00105-g14094bfeec09-aarch64-with-glibc2.35', 'value': '0.068'}, 'x': {'description': 'The simulated X data', 'units': 'kilograms', 'published_url': 'https://github.com/reproduce-work/reproduce-work/reproduce/pubdata.toml', 'generating_script': 'https://github.com/reproduce-work/reproduce-work/blob/main/nbs/00_introduction.ipynb', 'type': 'data', 'content_hash': '38f13b81a58a7d931600e917d77dfe8f', 'python_version': '3.9.13 | packaged by conda-forge | (main, May 27 2022, 17:23:2

## Test Code

In [12]:
# Test the serialize_to_toml function
data_sample = {
    'name': 'John',
    'age': 28,
    'is_student': False,
    'scores': [85, 90, 78, 92],
    'birthday': pd.Timestamp('2000-01-01'),
    'matrix': np.array([[1, 2], [3, 4]]),
    'df': pd.DataFrame({
        'A': [1, 2, 3],
        'B': ['a', 'b', 'c'],
        'date': [pd.Timestamp('2022-01-01'), pd.Timestamp('2022-01-02'), pd.Timestamp('2022-01-03')]
    }),
    'nested_dict': {
        'key1': 'value1',
        'sub_dict': {
            'sub_key': 'sub_value'
        }
    },
    'none_value': None
}

toml_representation = serialize_to_toml(data_sample)
printrw(f'{toml_representation}')

╔ω: name = "John"
age = 28
is_student = False
scores = [85, 90, 78, 92]
birthday = "2000-01-01 00:00:00"
[df]
[dataframe]
A = [1, 2, 3]
B = ["a", "b", "c"]
date = ["2022-01-01 00:00:00", "2022-01-02 00:00:00", "2022-01-03 00:00:00"]

[nested_dict]
key1 = "value1"
[sub_dict]
sub_key = "sub_value"


none_value = null
